In [3]:

import numpy as np
import pandas as pd

In [8]:
data_path="data/datafinal.json"
ftraind_path="data/f_traind.csv"

submission_path="data/submission.csv"

# vars
col_text='Text'
col_cc='ContainsCode'
col_cl='CodeList'


In [9]:
full_data=pd.read_json(data_path)
ftrain_data=pd.read_csv(ftraind_path)
train_data=full_data[full_data[col_cc]!=""]
test_data=full_data[full_data[col_cc]==""]
# pprint(train_data.iloc[1,1])
train_data
full_data.iloc[10]
# full_data.to_csv("/home/u131168/mh_shell/data/datafinal.csv")

ID                                                             11
Text            The field of software development is incredibl...
ContainsCode                                                 True
CodeList        #include <iostream> using namespace std; int m...
Name: 10, dtype: object

In [6]:
mh_dir='mh_shell'

In [12]:

!pip install bitsandbytes
!pip install accelerate
!pip install scipy
!pip install transformers
!pip install peft
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer
import torch
model_path=f"/home/u131168/{mh_dir}/ft_models/flan-t5-xl_peft_finetuned_model/checkpoint-6900"
# model_path="/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_ss/checkpoint-7000"

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
custom_model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_path, )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
from pprint import pprint

# p_count=50
psi,pei=50,60
prompts = ftrain_data.loc[psi:pei,['input','instruction']].values.tolist()
# instruction="extract code snippets from the paragraph, if it does not contain code snippets then say none"

t_prompts=[]
for p in prompts:
    context=str(p[0])
    question=p[1]
    t_prompts.append([context, question])
    # t_prompts+=[f"input: {context}\n\ninstruction: {question}"]

prompts=t_prompts
# prompts = ftrain_data.loc[:p_count,['input','instruction']].values.tolist()
# pprint(prompts)

In [14]:
input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=50)
pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
pred_custom_df=pd.DataFrame(pd.Series(pred_custom,name="pred_custom"))
# pred_ft5_df=pd.DataFrame(pd.Series(pred_ft5,name="pred_ft5"))
temp_df=pd.concat([ftrain_data.iloc[psi:pei].reset_index()],axis=1)
temp_df
# pprint(temp_df.iloc[1,0])

In [6]:
# # !pip install tiktoken
# # !pip install transformers
# # model='Salesforce/codegen-350M-mono'
# # model='Salesforce/codegen25-7b-instruct'
# model="Phind/Phind-CodeLlama-34B-v1"
# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model)


# # from transformers import AutoTokenizer, LlamaForCausalLM
# # model_path = "Phind/Phind-CodeLlama-34B-v1"
# # model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# # tokenizer = AutoTokenizer.from_pretrained(model_path)




ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [30]:
from pprint import pprint
# creating prompts
prompts=[]
for i in train_data[124:125].index:
    prompt='extract code from the following paragraph:\n\n\n'+train_data.iloc[i,1].replace('\n','')
    
    # predict
    # text = "def hello_world():"
    # text = prompt
    # input_ids = tokenizer(text, return_tensors="pt").input_ids
    # generated_ids = model.generate(input_ids, max_length=128)
    # res=tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

    # Generate
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=256, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
    completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    completion = completion.replace(prompt, "").split("\n\n\n")[0]
    res=completion

    print('-----------------------text----------------')
    pprint(prompt)
    print('-------------------------res------------------')
    pprint(res)

    


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-----------------------text----------------
('extract code from the following paragraph:\n'
 '\n'
 '\n'
 'DevOps is a set of practices that combines software development and IT '
 'operations. It aims to shorten the system development life cycle and provide '
 'continuous delivery with high software quality. DevOps is complementary with '
 'Agile software development; several DevOps aspects came from Agile '
 'methodology.')
-------------------------res------------------
('The problem is not a programming problem, but a text extraction problem. '
 'Here is a simple Python script that extracts the code from the given '
 'paragraph:\n'
 '\n'
 '```python\n'
 'paragraph = "DevOps is a set of practices that combines software development '
 'and IT operations. It aims to shorten the system development life cycle and '
 'provide continuous delivery with high software quality. DevOps is '
 'complementary with Agile software development; several DevOps aspects came '
 'from Agile methodology."\